In [25]:
import math
import sklearn
import numpy

import theano
import theano.tensor as T
import lasagne

import json

import pandas as pd

In [5]:
good = open("../scripts/codestyle_stats/fine_res.json", "r")
bad = open("../scripts/codestyle_stats/shit_res.json", "r")

In [6]:
bad_data = bad.read().replace('\n', ' ')
bad_dict = json.loads(bad_data)

In [7]:
good_data = good.read().replace('\n', ' ')
good_dict = json.loads(good_data)

In [8]:
print len(bad_dict)

608


In [22]:
print len(good_dict)

2540


In [33]:
X_train = []
for i in (1, len(bad_dict)):
    print bad_dict[i]["errors"]

{u'E901': 0, u'E227': 0, u'E721': 0, u'W604': 0, u'W603': 0, u'W602': 0, u'W601': 0, u'E224': 0, u'W503': 0, u'E101': 0, u'E501': 0, u'E711': 0, u'E201': 0, u'E203': 0, u'E202': 0, u'E128': 0, u'E129': 0, u'E124': 0, u'E125': 0, u'E126': 0, u'E127': 0, u'E121': 0, u'E122': 0, u'E123': 0, u'E266': 0, u'E265': 0, u'E228': 0, u'E402': 0, u'E261': 0, u'E223': 0, u'E222': 0, u'E302': 0, u'E303': 0, u'E304': 0, u'E226': 0, u'E306': 0, u'E262': 0, u'E401': 0, u'E701': 0, u'E221': 0, u'E115': 0, u'E114': 0, u'E116': 0, u'E111': 0, u'E113': 0, u'E112': 0, u'E731': 0, u'E271': 0, u'E305': 0, u'E241': 0, u'W291': 0, u'E714': 0, u'E301': 0, u'W293': 5, u'E211': 0, u'E713': 0, u'E502': 0, u'E131': 0, u'E712': 0, u'E274': 0, u'E275': 0, u'E251': 0, u'E225': 0, u'E272': 0, u'E273': 0, u'E704': 0, u'E231': 0, u'E242': 0, u'E191': 0, u'E722': 0, u'E702': 0, u'E703': 0}


IndexError: list index out of range

In [10]:
def build_network(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None, 1, 78),
                                     input_var=input_var)
    
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p = 0.2)
    
    l_hid = lasagne.layers.DenseLayer(l_in_drop, num_units=80, 
                                       nonlinearity=lasagne.nonlinearities.rectify,
                                       W=lasagne.init.GlorotUniform())
    l_out = lasagne.layers.DenseLayer(l_hid, num_units=2, 
                                      nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_out

In [11]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [17]:
input_var = T.tensor3('inputs')
target_var = T.ivector('targets')

In [18]:
network = build_network(input_var)

In [19]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, 
                                                   target_var)
loss = loss.mean()

In [20]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params,
                                            learning_rate=0.01,
                                            momentum=0.9)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(
                                                    test_prediction,
                                                    target_var)
test_loss = test_loss.mean()

In [21]:
train_fn = theano.function([input_var, target_var], loss,
                           updates=updates)
val_fn = theano.function([input_var, target_var], test_loss)

In [ ]:
for epoch in range(500):
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 100, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 100, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1
    print train_err / train_batches
    print val_err / val_batches